### clustering_lda_test_models
Tests many different versions of topic modeling using documents from the CPDOC_AAS database at MySQL.

In [1]:
import nltk
import os
import codecs
import string
import matplotlib.pyplot as plt
import matplotlib as mpl
from gensim import corpora, models, similarities #Latent Dirichlet Allocation implementation with Gensim
import pyLDAvis
import pyLDAvis.gensim
from IPython.display import clear_output

import sqlite3
import pickle

In [2]:
inputs = os.path.join("..", "inputs")
outputs = os.path.join("..","outputs")

### set user-specific variables
Verifies what operational system is being used and creates user-specific variables. Renato = Linux ; Marcelo = nt (Windows)

Also sets working folders

In [3]:
if os.name == 'nt':
    encoding_type = 'utf-8'
else:
    encoding_type = 'ISO-8859-1'

### selects texts from sql database to start topic modeling

In [4]:
sql_db = os.path.join(inputs, 'cpdoc_as.sqlite')
conn = sqlite3.connect(sql_db)
cur = conn.cursor()

count = 0
texts = []

'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
selects texts from sql database to start topic modeling
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
cur.execute("SELECT * FROM docs WHERE main_language = 'pt' AND (readability > 0.4 OR readability = -1) ") # filtra textos   
data = cur.fetchall()
numrows = len(data)
percentil = int(numrows/100)

for row in data:
    count += 1
    if row is None: break

    if count % percentil == 0: 
        clear_output()
        print(int(count/percentil),'% done')

    text =  row[4]
    text = text.split()
    symbols = [x for x in string.punctuation]
    text = [p for p in text if p not in symbols]
    text = [p.strip(string.punctuation) for p in text]
    text = [p for p in text if not p.isdigit()]
    text = [p for p in text if len(p)>1]
    texts.append(text)

100 % done


### makes list of stopwords

In [5]:
additional_words = ['mr','one', 'two', 'three', 'four', 
                    'five', 'um', 'dois', 'três', 'quatro', 
                    'cinco', 'janeiro', 'fevereiro', 'março', 
                    'abril', 'maio', 'junho', 'julho', 'agosto', 
                    'setembro', 'outubro', 'novembro', 'dezembro', 
                    'january', 'february', 'march', 'april', 'may', 
                    'june', 'july', 'august', 'september', 
                    'october', 'november', 'december', 'países', 
                    'ser', 'país', 'ainda', 'milhões', 'maior', 
                    'anos', 'grande', 'apenas', 'outros', 'pode', 
                    'parte', 'partes', 'item', 'vossa', 'nota', 
                    'havia', 'pt', 'vg', 'ptvg', 'eh', 'nr', 'hrs', 
                    'pais', 'parte', 'hoje', 'brasemb', 'ontem', 
                    'dia', 'countries', 'would', 'new', 'also', 
                    'must', 'draft', 'shall', 'item', 'page', 
                    'th', 'anos', 'ii', 'dias', 'poderá', 'caso', 
                    'casos', 'qualquer', 'ano', 'mil', 'pessoas', 
                    'único', 'única', 'únicos', 'únicas', 'índice', 
                    'expedido', 'co', 'mm', 'er', 'via', 'ww', 'ra', 
                    'ia', 'ca', 'nu', 'wa', 'aa', 'ms', 'dc', 'mmm', 'pa']

stopwords = nltk.corpus.stopwords.words('english') + \
            nltk.corpus.stopwords.words('portuguese') + \
            nltk.corpus.stopwords.words('french') + \
            nltk.corpus.stopwords.words('spanish') + \
            nltk.corpus.stopwords.words('german') + \
            additional_words

stopwords = list(set(stopwords))

# Topic Modeling
Many tests with Latent Dirichlet Allocation (LDA): 20, 30, 40, 45, 60 and 100 topic structure. After 60, started to set random_state 0 to avoid random variability between tests.

Also tests with Hierarchic Dirichlet Process (HDP).

In [6]:
%time texts = [[word for word in text if word not in stopwords] for text in texts]

Wall time: 2min 1s


In [7]:
print(len(texts[0]))

47


In [8]:
%%time
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=1, no_above=0.8)
dictionary.filter_tokens(bad_ids=[0,]) #retira palavras a partir do id
corpus = [dictionary.doc2bow(text) for text in texts]

In [9]:
len(corpus)

9054

#### saves pickle of dictionary and corpus

In [15]:
file_corpus = os.path.join(outer_outputs, 'LDAcorpus.pkl')
file_dictionary = os.path.join(outer_outputs, 'LDAdictionary.pkl')

In [5]:
''' caso queira carregar os arquivos '''
corpus = pickle.load(open(file_corpus, 'rb'))
dictionary = pickle.load(open(file_dictionary, 'rb'))

### tests with 20 topics

In [11]:
%time lda20 = models.LdaModel(corpus, num_topics=20, id2word=dictionary, passes=50, eval_every=1, random_state=0)
file = os.path.join(outputs, 'model_lda_20.pkl')
with open(os.path.join(outputs,'model_lda_20.pkl'), 'wb') as f:
    pickle.dump(lda20, f)

#### saves visualization of 20 topics

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda20, corpus, dictionary)

In [ ]:
data_ldavis = pyLDAvis.gensim.prepare(lda20, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output_20topics.html'))

### tests with 30 topics

In [ ]:
%time lda30 = models.LdaModel(corpus, num_topics=30, id2word=dictionary, passes=50, eval_every=1, random_state=0)
file = os.path.join('..', outputs, 'model_lda_30.pkl')
pickle.dump(lda30, open(file, 'wb'))

In [ ]:
lda30.print_topics(-1, num_words=5)

#### saves visualization of 30 topics

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda30, corpus, dictionary)

In [ ]:
data_ldavis = pyLDAvis.gensim.prepare(lda30, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output_30topics.html'))

### tests with 40 topics

In [ ]:
%time lda40 = models.LdaModel(corpus, num_topics=40, id2word=dictionary, passes=50, eval_every=1, random_state=0)
file = os.path.join('..', outputs, 'model_lda_40.pkl')
pickle.dump(lda40, open(file, 'wb'))

In [ ]:
lda40.print_topics(-1, num_words=5)

#### saves visualization of 40 topics

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda40, corpus, dictionary)

In [ ]:
data_ldavis = pyLDAvis.gensim.prepare(lda40, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output_40topics.html'))

### tests with 45 topics

In [6]:
%time lda45 = models.LdaModel(corpus, num_topics=45, id2word=dictionary, passes=50, eval_every=1, random_state=0)
file = os.path.join('..', outputs, 'model_lda_45.pkl')
pickle.dump(lda45, open(file, 'wb'))

CPU times: user 17h 8min 58s, sys: 3d 10h 13min 12s, total: 4d 3h 22min 11s
Wall time: 5h 19min 30s


In [7]:
lda45.print_topics(-1, num_words=5)

[(0,
  '0.018*comissão + 0.016*projeto + 0.015*geral + 0.012*resolução + 0.010*assembleia'),
 (1, '0.043*made + 0.041*use + 0.032*end + 0.029*ambassador + 0.026*aaaaa'),
 (2,
  '0.092*venezuela + 0.052*guiana + 0.028*caracas + 0.022*trinidad + 0.021*cuba'),
 (3,
  '0.013*política + 0.012*desenvolvimento + 0.010*relações + 0.010*brasil + 0.009*internacional'),
 (4,
  '0.026*brasil + 0.020*cooperação + 0.013*acordo + 0.009*desenvolvimento + 0.007*relações'),
 (5,
  '0.035*excelência + 0.031*brasil + 0.031*senhor + 0.023*ministro + 0.020*presidente'),
 (6, '0.005*mw + 0.005*dn + 0.005*ar + 0.004*mu + 0.004*lu'),
 (7,
  '0.063*nuclear + 0.030*acordo + 0.022*energia + 0.022*nucleares + 0.021*brasil'),
 (8, '0.008*ro + 0.008*ue + 0.007*rr + 0.007*ão + 0.006*ar'),
 (9,
  '0.069*global + 0.026*ministro + 0.025*exteriores + 0.024*york + 0.023*retransmito'),
 (10,
  '0.019*brasil + 0.014*visto + 0.011*estrangeiro + 0.010*art + 0.009*brasileiro'),
 (11,
  '0.023*cuba + 0.021*uruguai + 0.019*gobie

#### saves visualization of 45 topics

In [8]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda45, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
21     13.283769        1       1  0.187327 -0.153759
44      9.403938        1       2  0.190644 -0.017513
3       8.166329        1       3  0.242041 -0.166630
19      5.983419        1       4  0.096709 -0.150007
0       5.237096        1       5  0.185763 -0.108180
4       5.025599        1       6  0.245962 -0.067651
13      3.518223        1       7  0.161206  0.085080
22      3.292145        1       8  0.132172 -0.152160
17      3.244874        1       9  0.135468 -0.092046
33      2.927655        1      10  0.159345 -0.127406
35      2.612802        1      11  0.003168  0.189011
15      2.582286        1      12  0.171185 -0.080576
5       2.548495        1      13  0.196660  0.144392
38      2.394476        1      14 -0.112496 -0.036090
39      2.332376        1      15  0.166407  0.132030
34      2.247589        1      16  0.138673  0.182874
42      2.233285        1      17  0.071107 -0.061775
43      2.191938        1      18  0.138888 -0.061182
12      2.061485        1      19  0.013927  0.161076
6       2.016087        1      20 -0.074005  0.054780
8       1.749013        1      21 -0.095541  0.025601
7       1.642205        1      22  0.105726 -0.090930
28      1.534915        1      23 -0.009118  0.119463
10      1.239526        1      24  0.036559 -0.015696
27      1.182871        1      25  0.050745  0.226474
32      1.068028        1      26  0.015927  0.250977
25      1.037455        1      27 -0.041161 -0.005991
37      0.982759        1      28  0.036301  0.067805
14      0.865740        1      29 -0.098771  0.060386
23      0.727719        1      30  0.004860  0.281328
11      0.706471        1      31 -0.086318  0.044169
9       0.688439        1      32 -0.097900  0.126172
31      0.502780        1      33 -0.169596  0.002548
26      0.471575        1      34 -0.002603 -0.129711
24      0.469602        1      35 -0.149888 -0.050381
41      0.396422        1      36 -0.155447  0.017110
2       0.258450        1      37 -0.162092 -0.039934
18      0.222032        1      38 -0.184926 -0.059035
29      0.199211        1      39 -0.211853 -0.062133
30      0.172691        1      40 -0.188989 -0.059006
40      0.164226        1      41 -0.188311 -0.085431
16      0.137284        1      42 -0.198866 -0.046889
1       0.123656        1      43 -0.216898 -0.072447
36      0.123508        1      44 -0.216401 -0.072620
20      0.029555        1      45 -0.225590 -0.106099, topic_info=      Category          Freq               Term         Total  loglift  \
term                                                                     
8613   Default  21602.000000         presidente  21602.000000  30.0000   
41727  Default  18636.000000             senhor  18636.000000  29.0000   
36603  Default  13979.000000          telegrama  13979.000000  28.0000   
1614   Default  21361.000000           ministro  21361.000000  27.0000   
72263  Default  49020.000000             brasil  49020.000000  26.0000   
15517  Default  12543.000000         excelência  12543.000000  25.0000   
79392  Default  23183.000000           relações  23183.000000  24.0000   
18942  Default  16381.000000         exteriores  16381.000000  23.0000   
35295  Default   9339.000000             visita   9339.000000  22.0000   
31287  Default  23249.000000            governo  23249.000000  21.0000   
47101  Default   5836.000000            nuclear   5836.000000  20.0000   
18996  Default   6918.000000            secreto   6918.000000  19.0000   
91224  Default  12869.000000             acordo  12869.000000  18.0000   
4669   Default  10792.000000         embaixador  10792.000000  17.0000   
10821  Default  12420.000000          república  12420.000000  16.0000   
12216  Default   5802.000000             itaipu   5802.000000  15.0000   
31000  Default   6489.000000             artigo   6489.000000  14.0000   
26083  Defaul

In [9]:
data_ldavis = pyLDAvis.gensim.prepare(lda45, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output_45topics.html'))

### tests with 60 topics and random_state 0

In [10]:
%time lda60_00 = models.LdaModel(corpus, num_topics=60, id2word=dictionary, passes=50, eval_every=1, random_state=0)
file = os.path.join('..', outputs, 'model_lda_60_rs_00.pkl')
pickle.dump(lda60_00, open(file, 'wb'))

CPU times: user 18h 39min 2s, sys: 3d 17h 48min 24s, total: 4d 12h 27min 27s
Wall time: 5h 43min 15s


#### saves visualization of 60 topics

In [11]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda60_00, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
21     12.583668        1       1  0.196208 -0.129793
26      8.145018        1       2  0.267420 -0.196985
44      7.801926        1       3  0.236515 -0.042959
19      5.428203        1       4  0.190646 -0.164644
42      4.568603        1       5  0.257102  0.004746
33      4.254083        1       6  0.197826 -0.073142
15      4.240773        1       7  0.189190 -0.064421
3       3.138811        1       8  0.192975 -0.135690
46      2.930916        1       9  0.117869 -0.130304
0       2.879378        1      10  0.214439 -0.041750
4       2.840419        1      11  0.216318  0.014366
39      2.737194        1      12  0.193965  0.088021
54      2.304393        1      13  0.121975  0.147904
35      2.268367        1      14  0.098013  0.243274
18      2.051364        1      15  0.190500 -0.084062
8       1.880394        1      16 -0.051702  0.044564
49      1.844679        1      17  0.095356 -0.089888
51      1.728295        1      18  0.053622 -0.010014
28      1.693466        1      19  0.082125  0.213373
30      1.643145        1      20  0.165299 -0.088371
13      1.310708        1      21 -0.090643  0.005723
32      1.293606        1      22  0.079363  0.215428
6       1.287538        1      23  0.020598  0.122542
41      1.237523        1      24 -0.025841  0.111426
50      1.225994        1      25 -0.091711 -0.028688
36      1.197033        1      26 -0.062136 -0.047899
56      1.193781        1      27  0.048332 -0.038030
34      1.189762        1      28  0.082676  0.137534
12      1.100863        1      29 -0.001445  0.207357
38      1.095758        1      30 -0.041893  0.086092
53      1.037696        1      31  0.035949  0.293666
37      0.976501        1      32  0.084457  0.183922
52      0.804547        1      33 -0.089073  0.011869
40      0.632058        1      34 -0.101825 -0.017899
43      0.600222        1      35 -0.031640  0.053180
47      0.562991        1      36 -0.105782  0.005389
27      0.552651        1      37 -0.069163  0.121732
11      0.540486        1      38 -0.046383  0.057482
10      0.461031        1      39 -0.017361  0.026732
17      0.459630        1      40 -0.005674 -0.023061
25      0.427809        1      41 -0.104159 -0.036791
45      0.351680        1      42 -0.133487 -0.022885
14      0.335760        1      43 -0.143447 -0.008366
29      0.329056        1      44 -0.141951 -0.024681
9       0.328603        1      45 -0.130223  0.011073
31      0.320180        1      46 -0.080529 -0.008215
23      0.294582        1      47 -0.152289 -0.016238
22      0.252187        1      48  0.094700 -0.176897
1       0.212465        1      49 -0.136680 -0.062174
2       0.201634        1      50 -0.151659 -0.046864
16      0.177180        1      51 -0.152473 -0.045855
59      0.170974        1      52 -0.152334 -0.035988
48      0.141973        1      53 -0.179096 -0.058788
7       0.137380        1      54 -0.159178 -0.041918
20      0.132579        1      55 -0.176146 -0.068806
58      0.123820        1      56 -0.172849 -0.057886
5       0.118918        1      57 -0.160976 -0.033742
57      0.115586        1      58 -0.179277 -0.058930
55      0.074585        1      59 -0.189788 -0.084532
24      0.029576        1      60 -0.194623 -0.110237, topic_info=      Category          Freq         Term         Total  loglift  logprob
term                                                                     
72263  Default  49027.000000       brasil  49027.000000  30.0000  30.0000
8613   Default  20934.000000   presidente  20934.000000  29.0000  29.0000
36603  Default  13258.000000    telegrama  13258.000000  28.0000  28.0000
41727  Default  19223.000000       senhor  19223.000000  27.0000  27.0000
56958  Default   8698.000000       unidos   8698.000000  26.0000  26.0000
1614   Default  21615.000000     ministro  21615.000000  25.0000  25.0000
15517  Default  1

In [12]:
data_ldavis = pyLDAvis.gensim.prepare(lda60_00, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output_60topics_rs00.html'))

### tests with 60 topics and random_state 1

In [13]:
%time lda60_01 = models.LdaModel(corpus, num_topics=60, id2word=dictionary, passes=50, eval_every=1, random_state=1)
file = os.path.join('..', outputs, 'model_lda_60_rs_01.pkl')
pickle.dump(lda60_01, open(file, 'wb'))

CPU times: user 18h 44min 14s, sys: 3d 18h 1min 1s, total: 4d 12h 45min 15s
Wall time: 5h 43min 31s


#### saves visualization of 60 topics

In [14]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda60_01, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
17     10.384952        1       1 -0.159351  0.150194
23      9.480886        1       2 -0.165872  0.173591
39      6.868180        1       3 -0.242921  0.118641
51      6.627089        1       4 -0.252827  0.150228
50      5.910252        1       5 -0.252859 -0.049571
29      5.476138        1       6 -0.216138  0.214557
36      4.195866        1       7 -0.240180  0.042891
15      3.398514        1       8 -0.172906  0.059281
19      3.338780        1       9 -0.031434 -0.074564
20      3.181606        1      10 -0.196158  0.150423
13      2.624479        1      11 -0.104581 -0.239723
2       2.529585        1      12 -0.189823  0.053592
57      2.491463        1      13 -0.147500 -0.085612
7       2.100830        1      14 -0.161124 -0.231218
18      1.984258        1      15 -0.097467 -0.039151
48      1.840611        1      16 -0.039655 -0.133885
32      1.757196        1      17 -0.168862  0.130060
3       1.582503        1      18  0.077654  0.010733
44      1.528269        1      19 -0.118154 -0.171580
38      1.450319        1      20 -0.135773  0.122089
9       1.438128        1      21 -0.068807 -0.094071
0       1.396578        1      22 -0.113074  0.002942
25      1.373312        1      23  0.039752  0.013016
56      1.367173        1      24  0.037557 -0.114346
28      1.256025        1      25 -0.051717 -0.115748
49      1.137591        1      26 -0.131182  0.141275
14      1.085053        1      27 -0.064339 -0.105477
1       1.019557        1      28 -0.064951  0.088759
11      0.960739        1      29 -0.104960 -0.031659
4       0.946832        1      30 -0.024029 -0.000353
43      0.885169        1      31 -0.054741 -0.254062
47      0.827186        1      32 -0.066720 -0.132881
42      0.689704        1      33  0.001379 -0.038089
31      0.664573        1      34  0.033955  0.018489
35      0.664001        1      35  0.025245 -0.107350
59      0.583187        1      36  0.029453 -0.014367
26      0.505869        1      37  0.085335 -0.064561
40      0.498003        1      38  0.012357  0.092855
12      0.472982        1      39  0.049261 -0.187933
8       0.461343        1      40 -0.040415 -0.107174
34      0.404411        1      41  0.119961 -0.017400
55      0.351634        1      42  0.115945 -0.036184
10      0.330904        1      43  0.112780 -0.013302
21      0.288817        1      44  0.089337  0.010384
16      0.185023        1      45  0.144149 -0.010992
33      0.175405        1      46  0.140728  0.001700
54      0.172081        1      47  0.169704  0.023715
45      0.138561        1      48  0.171101  0.024494
52      0.117938        1      49  0.189961  0.038751
27      0.114323        1      50  0.166794  0.052119
58      0.111375        1      51  0.188677  0.036411
30      0.107007        1      52  0.189133  0.038973
37      0.084601        1      53  0.188338  0.046938
6       0.084360        1      54  0.188006  0.043960
41      0.070714        1      55  0.208693  0.056190
53      0.069808        1      56  0.214003  0.067299
5       0.054093        1      57  0.220125  0.071325
46      0.053017        1      58  0.223862  0.074968
24      0.050604        1      59  0.223075  0.075221
22      0.050546        1      60  0.222203  0.075191, topic_info=      Category          Freq           Term         Total  loglift  logprob
term                                                                       
72263  Default  49100.000000         brasil  49100.000000  30.0000  30.0000
41727  Default  19446.000000         senhor  19446.000000  29.0000  29.0000
1614   Default  21801.000000       ministro  21801.000000  28.0000  28.0000
8613   Default  21682.000000     presidente  21682.000000  27.0000  27.0000
36603  Default  12845.000000      telegrama  12845.000000  26.0000  26.0000
15517  Default  12847.000000     excelência  12847.000000  25.0000  25.0000
7

In [15]:
data_ldavis = pyLDAvis.gensim.prepare(lda60_01, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output_60topics_rs01.html'))

### tests with 100 topics and random_state 0

In [18]:
%time lda100_00 = models.LdaModel(corpus, num_topics=100, id2word=dictionary, passes=50, eval_every=1, random_state=0)
file = os.path.join('..', outputs, 'model_lda_100_rs_00..pkl')
pickle.dump(lda100_00, open(file, 'wb'))

Wall time: 57min 27s


#### saves visualization of 100 topics

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda100_00, corpus, dictionary)

In [ ]:
data_ldavis = pyLDAvis.gensim.prepare(lda100_00, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outer_outputs,'pyldavis_output_100topics_rs00.html'))

### tests with Hierarchic Dirichlet Process (HDP)

In [19]:
%time hdp = models.HdpModel(corpus, id2word=dictionary)

CPU times: user 5min 19s, sys: 23min 55s, total: 29min 14s
Wall time: 2min


In [20]:
hdp.print_topics()

['topic 0: 0.009*brasil + 0.004*relações + 0.004*governo + 0.004*presidente + 0.004*ministro + 0.004*senhor + 0.003*exteriores + 0.002*política + 0.002*acordo + 0.002*brasileiro + 0.002*telegrama + 0.002*excelência + 0.002*república + 0.002*desenvolvimento + 0.002*embaixador + 0.002*cooperação + 0.002*internacional + 0.002*rio + 0.002*brasileira + 0.002*visita',
 'topic 1: 0.006*brasil + 0.004*política + 0.003*governo + 0.003*ministro + 0.003*guerra + 0.003*senhor + 0.003*desenvolvimento + 0.003*mundo + 0.003*marinha + 0.003*unidos + 0.002*relações + 0.002*nacional + 0.002*poder + 0.002*internacional + 0.002*cooperação + 0.002*segurança + 0.002*militar + 0.001*estratégia + 0.001*cada + 0.001*silveira',
 'topic 2: 0.006*brasil + 0.002*política + 0.002*nacional + 0.002*itamaraty + 0.002*brasileira + 0.002*diplomacia + 0.002*governo + 0.001*exterior + 0.001*desenvolvimento + 0.001*presidente + 0.001*exteriores + 0.001*relações + 0.001*senhor + 0.001*externa + 0.001*nações + 0.001*mundo + 

In [21]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(hdp, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      92.158650        1       1  0.336527 -0.023703
1       3.338865        1       2  0.294153 -0.072489
4       1.053975        1       3  0.063214  0.064573
3       0.808857        1       4  0.119264  0.083099
2       0.436554        1       5  0.136693 -0.059907
5       0.291100        1       6  0.060966  0.059085
8       0.208929        1       7  0.022105  0.045450
7       0.122424        1       8  0.023881  0.050156
6       0.120007        1       9  0.036030  0.045771
26      0.103219        1      10  0.000955 -0.006434
9       0.076210        1      11  0.020777  0.047847
13      0.075749        1      12  0.013678  0.035659
10      0.055061        1      13  0.019375  0.044791
12      0.050442        1      14  0.018435 -0.012033
11      0.038636        1      15  0.018091  0.028645
19      0.037437        1      16  0.006928  0.022997
15      0.036364        1      17  0.014494  0.022626
17      0.035342        1      18  0.009934  0.031040
16      0.033716        1      19  0.015269  0.019849
14      0.028968        1      20  0.020449  0.015277
23      0.028813        1      21  0.012151  0.010521
28      0.027503        1      22  0.005529 -0.000782
20      0.026953        1      23  0.005414  0.028994
74      0.024746        1      24 -0.012484 -0.004446
22      0.022717        1      25  0.006262  0.023376
21      0.020345        1      26  0.009536  0.019733
25      0.020330        1      27  0.006870  0.021084
18      0.017496        1      28  0.012519  0.020773
35      0.017343        1      29 -0.000365  0.010602
32      0.017189        1      30 -0.001774  0.005829
...          ...      ...     ...       ...       ...
108     0.002386        1     121 -0.015623 -0.008979
124     0.002328        1     122 -0.015523 -0.007999
113     0.002198        1     123 -0.015065 -0.009471
82      0.002117        1     124 -0.015646 -0.010541
98      0.002097        1     125 -0.016040 -0.010603
137     0.001735        1     126 -0.015522 -0.012277
76      0.001665        1     127 -0.010007 -0.000071
109     0.001604        1     128 -0.016566 -0.010170
122     0.001588        1     129 -0.016442 -0.011416
145     0.001572        1     130 -0.016325 -0.011429
57      0.001531        1     131 -0.008367 -0.001260
142     0.001457        1     132 -0.017096 -0.010507
140     0.001436        1     133 -0.016061 -0.008898
115     0.001363        1     134 -0.016424 -0.009500
116     0.001357        1     135 -0.014600 -0.009910
86      0.001272        1     136 -0.010160 -0.006194
148     0.001261        1     137 -0.015416 -0.009855
81      0.001035        1     138 -0.013843 -0.008903
121     0.000880        1     139 -0.014517 -0.010713
102     0.000873        1     140 -0.015554 -0.010307
96      0.000872        1     141 -0.015827 -0.009333
101     0.000700        1     142 -0.013041 -0.005686
87      0.000645        1     143 -0.014809 -0.007029
129     0.000642        1     144 -0.015592 -0.008745
85      0.000427        1     145 -0.015435 -0.009271
135     0.000364        1     146 -0.015050 -0.010806
125     0.000344        1     147 -0.016735 -0.011157
133     0.000308        1     148 -0.016863 -0.011209
107     0.000015        1     149 -0.014318 -0.009473
147     0.000005        1     150 -0.014308 -0.011123

[150 rows x 5 columns], topic_info=       Category          Freq             Term         Total  loglift  logprob
term                                                                          
72263   Default  47150.000000           brasil  47150.000000  30.0000  30.0000
79392   Default  22682.000000         relações  22682.000000  29.0000  29.0000
1614    Default  20855.000000         ministro  20855.000000  28.0000  28.0000
41727   Default  19437.000000           senhor  19437.000000  27.0000  27.0000
18942   Default  15540.000000       exteriores  15540.000

In [22]:
data_ldavis = pyLDAvis.gensim.prepare(hdp, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output_hdp.html'))